In [6]:
import pandas as pd

In [7]:
df_fact = pd.read_csv('Fact_TrafficViolation.csv')
df_dim_date = pd.read_csv('Dim_Date.csv')
df_external = pd.read_csv('External_Bicycle.csv')

In [8]:
df_dim_date['ViolationDate'] = pd.to_datetime(df_dim_date['Date']).dt.date

df_fact_dated = pd.merge(
    df_fact,
    df_dim_date[['DateKey', 'ViolationDate']],
    on='DateKey',
    how='left'
)

In [9]:
df_external['starttime_dt'] = pd.to_datetime(df_external['starttime'])
df_external['BicycleDate'] = df_external['starttime_dt'].dt.date

df_external['Customer_Flag'] = (df_external['usertype'] == 'Customer').astype(int)
df_external['Subscriber_Flag'] = (df_external['usertype'] == 'Subscriber').astype(int)

df_external['Male_Flag'] = (df_external['gender'] == 'Male').astype(int)
df_external['Female_Flag'] = (df_external['gender'] == 'Female').astype(int)

# Group by date and calculate all required aggregations
df_bicycle_agg = df_external.groupby('BicycleDate').agg(
    UserType_Customer_Count=('Customer_Flag', 'sum'),
    UserType_Subscriber_Count=('Subscriber_Flag', 'sum'),
    Gender_Male_Count=('Male_Flag', 'sum'),
    Gender_Female_Count=('Female_Flag', 'sum'),
    # Sum of trip duration by date
    TotalTripDuration=('tripduration', 'sum')
).reset_index()

In [11]:
df_integrated_final = pd.merge(
    df_fact_dated,
    df_bicycle_agg,
    left_on='ViolationDate',
    right_on='BicycleDate',
    how='left'
)

count_cols = [
    'UserType_Customer_Count',
    'UserType_Subscriber_Count',
    'Gender_Male_Count',
    'Gender_Female_Count',
    'TotalTripDuration'
]

In [12]:
# Replace NaN values (where no bicycle data exists) with 0
na_records = df_integrated_final[df_integrated_final[count_cols].isna().any(axis=1)]
print(na_records)

        DateKey  WeatherKey  AddressKey  RedLightViolations  \
572         116         116         122                  31   
2956        116         116         297                   0   
3192        116         116         160                   1   
3633        116         116         333                   0   
5956        116         116         263                   1   
...         ...         ...         ...                 ...   
322300      116         116         352                   0   
323069      116         116         150                  10   
324497      116         116          96                   1   
324671      116         116         202                  12   
324690      116         116         281                   4   

        SpeedingViolations  TotalViolations                CreateTimeStamp  \
572                      0               31  2025-06-27 21:18:40.430000000   
2956                     4                4  2025-06-27 21:18:40.430000000   
3192     

In [13]:
columns_to_keep = list(df_fact.columns) + count_cols
df_final_output = df_integrated_final[columns_to_keep].copy()

output_filename = 'Integrated_Weather_Bicycle.csv'
df_final_output.to_csv(output_filename, index=False)

In [14]:
df_integrated = pd.read_csv('Integrated_Weather_Bicycle.csv')
df_integrated.head(5)

,DateKey,WeatherKey,AddressKey,RedLightViolations,SpeedingViolations,TotalViolations,CreateTimeStamp,SourceSystemCode,SourceFolder,UserType_Customer_Count,UserType_Subscriber_Count,Gender_Male_Count,Gender_Female_Count,TotalTripDuration
0,795,795,241,15,0,15,2025-06-27 21:18:40.430000000,2,2017-09-30,7.0,12086.0,8987.0,3106.0,144573.516667
1,1085,1085,223,2,0,2,2025-06-27 21:18:40.430000000,2,2017-09-30,7.0,12775.0,9570.0,3212.0,150680.300000
2,317,317,245,5,0,5,2025-06-27 21:18:40.430000000,2,2017-09-30,0.0,6933.0,5417.0,1516.0,75698.633333
3,1167,1167,206,3,0,3,2025-06-27 21:18:40.430000000,2,2017-09-30,0.0,9614.0,6611.0,3003.0,123137.883333
4,182,182,34,3,0,3,2025-06-27 21:18:40.430000000,2,2017-09-30,0.0,2385.0,1974.0,411.0,22106.900000
